In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import TextLoader
from langchain.output_parsers import StructuredOutputParser
from langchain.output_parsers import ResponseSchema

In [12]:
loader = TextLoader("./data.txt")
docs = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs) #if we need to split doc.
vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [13]:
contextualize_q_system_prompt = """Imagine you are a receptionist at South Bay Dental Office. """
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

qa_system_prompt = """Imagine you are a receptionist at South Bay Dental Office. \
    {context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever
    )
    | qa_prompt
    | llm
)

In [15]:
chat_history = []

for i in range(6):

  question = input("Himalay: ")
  ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
  print("AI Response: ",ai_msg)
  print("--------------------------------------------------------------")
  chat_history.extend([HumanMessage(content=question), ai_msg])

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


AI Response:  content='Hi, this is South Bay Dental Office. How may I help you?'
--------------------------------------------------------------


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


AI Response:  content='Sure, are you a new patient or an existing patient?'
--------------------------------------------------------------


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


...

AI Response:  content='Thank you for letting me know. In order to schedule an appointment for you, I will need your phone number and name. Could you please provide me with that information?'
--------------------------------------------------------------


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


AI Response:  content="Thank you, Himalay, for providing your information. Now, let's find a suitable appointment time for you. Our office hours are Monday to Friday from 9:00 AM to 6:00 PM, and Saturday from 9:00 AM to 1:00 PM. Could you please let me know your availability?"
--------------------------------------------------------------


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


AI Response:  content="I'm sorry, but we don't have any available appointments at 3:00 PM on Saturdays. However, we do have availability at 2:00 PM or 4:00 PM. Which one would work best for you?"
--------------------------------------------------------------
